In [1]:
%load_ext autoreload
%autoreload 2
import glob
import numpy as np
from sklearn.metrics import r2_score
from sklearn import preprocessing
import xgboost as xgb
from collections import defaultdict
from matplotlib import pyplot as plt
import pandas as pd
from itertools import product
import sys
sys.path.append('../genotype')
from data import GenotypeDataModule
np.random.seed(42)

In [2]:
path_pattern = "../datasets/genotype/cas9/cas9_pairs_10nm_%s.csv"

In [3]:
gene = "GACGCATAAAGATGAGACGCTGG"
pair2int = {pair: idx for  idx, pair in enumerate(product(['A', 'C', 'G', 'T'], ['A', 'C', 'G', 'T']))}
int2pair = {idx: pair for idx, pair in enumerate(product(['A', 'C', 'G', 'T'], ['A', 'C', 'G', 'T']))}

In [4]:
pp = path_pattern 
dm = GenotypeDataModule(paths = [pp%"train", pp%"valid", pp%"test"])

In [5]:
dm.prepare_data()
dm.setup()

In [6]:
X_train = dm.X_train.numpy()
X_valid = dm.X_valid.numpy()
X_test = dm.X_test.numpy()
Y_train = dm.y_train.numpy()[:, 0]
Y_valid = dm.y_valid.numpy()[:, 0]
Y_test  = dm.y_test.numpy()[:, 0]

In [7]:
def to_binary(X):
    mask0 = (X == pair2int[('A', 'A')]) + (X == pair2int[('C', 'C')]) + (X == pair2int[('G', 'G')]) + (X == pair2int[('T', 'T')])
    binary = X.copy()
    binary[mask0] = 0
    binary[~mask0] = 1
    return binary

In [8]:
X_train_binary, X_valid_binary, X_test_binary = to_binary(X_train), to_binary(X_valid), to_binary(X_test)

# XGBoost pairs

In [9]:
from sklearn.model_selection import PredefinedSplit, RandomizedSearchCV

In [10]:
param_grid = {'gamma': [0,0.1,0.2,0.4,0.8,1.6,3.2,6.4,12.8,25.6,51.2,102.4, 200],
              'learning_rate': [0.01, 0.03, 0.06, 0.1, 0.15, 0.2, 0.25, 0.300000012, 0.4, 0.5, 0.6, 0.7],
              'max_depth': [5,6,7,8,9,10,11,12,13,14],
              'n_estimators': [25,50,65,80,100,115,130,150,200,400,800],
              'reg_alpha': [0,0.1,0.2,0.4,0.8,1.6,3.2,6.4,12.8,25.6,51.2,102.4,200],
              'reg_lambda': [0,0.1,0.2,0.4,0.8,1.6,3.2,6.4,12.8,25.6,51.2,102.4,200],
              'subsample': [0.6, 0.8, 1.0],
              'colsample_bytree': [0.6, 0.8, 1.0]}

In [11]:
X_train_valid = np.concatenate((X_train, X_valid), axis=0)
Y_train_valid = np.concatenate((Y_train, Y_valid), axis=0)
fold = np.zeros(Y_train_valid.shape, dtype=np.int32)
fold[:len(X_train)] = -1
ps = PredefinedSplit(fold)

for train_index, test_index in ps.split():
    print(train_index)
    print(test_index)
    print()

In [ ]:
cv = RandomizedSearchCV(xgb.XGBRegressor(), param_grid, n_iter=1000, cv=ps)
cv.fit(X_train_valid, Y_train_valid)

In [ ]:
cv.best_params_

In [ ]:
est = xgb.XGBRegressor(**cv.best_params_)
est.fit(X_train, Y_train)
Y_pred = est.predict(X_test)
Y_pred2 = est.predict(X_train)
r2_train = r2_score(Y_train, Y_pred2)
rel_train = np.linalg.norm(Y_train - Y_pred2)/np.linalg.norm(Y_train)
r2 = r2_score(Y_test, Y_pred)
rel = np.linalg.norm(Y_test - Y_pred)/np.linalg.norm(Y_test)
print('TRAIN  R2 score: %2.4f, relative error: %2.4f'%(r2_train, rel_train))
print('TEST   R2 score: %2.4f, relative error: %2.4f'%(r2, rel))

In [ ]:
#{'subsample': 1.0,
# 'reg_lambda': 102.4,
# 'reg_alpha': 0,
# 'n_estimators': 100,
# 'max_depth': 14,
# 'learning_rate': 0.25,
# 'gamma': 1.6,
# 'colsample_bytree': 0.8}

#TRAIN  R2 score: 0.4616, relative error: 0.7337
#TEST   R2 score: 0.3989, relative error: 0.7729


# XGBoost binary

In [ ]:
X_train_valid_binary = np.concatenate((X_train_binary, X_valid_binary), axis=0)
cv = RandomizedSearchCV(xgb.XGBRegressor(), param_grid, n_iter=1000, cv=ps)
cv.fit(X_train_valid_binary, Y_train_valid)
print(cv.best_params_)

In [ ]:
est = xgb.XGBRegressor(**cv.best_params_)
est.fit(X_train_binary, Y_train)
Y_pred = est.predict(X_test_binary)
Y_pred2 = est.predict(X_train_binary)
r2_train = r2_score(Y_train, Y_pred2)
rel_train = np.linalg.norm(Y_train - Y_pred2)/np.linalg.norm(Y_train)
r2 = r2_score(Y_test, Y_pred)
rel = np.linalg.norm(Y_test - Y_pred)/np.linalg.norm(Y_test)
print('TRAIN  R2 score: %2.4f, relative error: %2.4f'%(r2_train, rel_train))
print('TEST   R2 score: %2.4f, relative error: %2.4f'%(r2, rel))

In [ ]:
#{'subsample': 1.0, 'reg_lambda': 25.6, 'reg_alpha': 0.8, 'n_estimators': 400, 'max_depth': 14, 'learning_rate': 0.300000012, 'gamma': 0.8, 'colsample_bytree': 0.8}
#TRAIN  R2 score: 0.6008, relative error: 0.6318
#TEST   R2 score: 0.4943, relative error: 0.7089

In [ ]:
X = X_train_binary.copy()

# XGBoost Poly

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
feats = PolynomialFeatures(degree=2)

In [ ]:
X_train_poly = feats.fit_transform(X_train_binary)
X_valid_poly = feats.fit_transform(X_valid_binary)
X_test_poly = feats.fit_transform(X_test_binary)

In [ ]:
X_train_valid_poly = np.concatenate((X_train_poly, X_valid_poly), axis=0)
cv = RandomizedSearchCV(xgb.XGBRegressor(), param_grid, n_iter=1000, cv=ps)
cv.fit(X_train_valid_poly, Y_train_valid)
print(cv.best_params_)

In [ ]:
est = xgb.XGBRegressor(**cv.best_params_)
est.fit(X_train_poly, Y_train)
Y_pred = est.predict(X_test_poly)
Y_pred2 = est.predict(X_train_poly)
r2_train = r2_score(Y_train, Y_pred2)
rel_train = np.linalg.norm(Y_train - Y_pred2)/np.linalg.norm(Y_train)
r2 = r2_score(Y_test, Y_pred)
rel = np.linalg.norm(Y_test - Y_pred)/np.linalg.norm(Y_test)
print('TRAIN  R2 score: %2.4f, relative error: %2.4f'%(r2_train, rel_train))
print('TEST   R2 score: %2.4f, relative error: %2.4f'%(r2, rel))

In [ ]:
# {'subsample': 0.8, 'reg_lambda': 0.8, 'reg_alpha': 12.8, 'n_estimators': 400, 'max_depth': 14, 'learning_rate': 0.2, 'gamma': 0.1, 'colsample_bytree': 0.6}
# TRAIN  R2 score: 0.5954, relative error: 0.6361
# TEST   R2 score: 0.4878, relative error: 0.7134

In [ ]:
from sklearn.linear_model import ElasticNetCV

In [ ]:
feats = PolynomialFeatures(degree=3)
X_train_poly = feats.fit_transform(X_train_binary)
X_valid_poly = feats.fit_transform(X_valid_binary)
X_test_poly = feats.fit_transform(X_test_binary)
X_train_valid_poly = np.concatenate((X_train_poly, X_valid_poly), axis=0)

est = ElasticNetCV(cv=ps, max_iter=10000)
est.fit(X_train_valid_poly, Y_train_valid)
print('alpha', est.alpha_)
print('l1 ratio', est.l1_ratio_)
Y_pred = est.predict(X_test_poly)
Y_pred2 = est.predict(X_train_poly)
r2_train = r2_score(Y_train, Y_pred2)
rel_train = np.linalg.norm(Y_train - Y_pred2)/np.linalg.norm(Y_train)
r2 = r2_score(Y_test, Y_pred)
rel = np.linalg.norm(Y_test - Y_pred)/np.linalg.norm(Y_test)
print('TRAIN  R2 score: %2.4f, relative error: %2.4f'%(r2_train, rel_train))
print('TEST   R2 score: %2.4f, relative error: %2.4f'%(r2, rel))

In [ ]:
#alpha 0.0004776418992889931
#l1 ratio 0.5
#TRAIN  R2 score: 0.4645, relative error: 0.7317
#TEST   R2 score: 0.2051, relative error: 0.8888

In [ ]:
feats = PolynomialFeatures(degree=2)
X_train_poly = feats.fit_transform(X_train_binary)
X_valid_poly = feats.fit_transform(X_valid_binary)
X_test_poly = feats.fit_transform(X_test_binary)
X_train_valid_poly = np.concatenate((X_train_poly, X_valid_poly), axis=0)

est = ElasticNetCV(cv=ps, max_iter=10000)
est.fit(X_train_valid_poly, Y_train_valid)
print('alpha', est.alpha_)
print('l1 ratio', est.l1_ratio_)
Y_pred = est.predict(X_test_poly)
Y_pred2 = est.predict(X_train_poly)
r2_train = r2_score(Y_train, Y_pred2)
rel_train = np.linalg.norm(Y_train - Y_pred2)/np.linalg.norm(Y_train)
r2 = r2_score(Y_test, Y_pred)
rel = np.linalg.norm(Y_test - Y_pred)/np.linalg.norm(Y_test)
print('TRAIN  R2 score: %2.4f, relative error: %2.4f'%(r2_train, rel_train))
print('TEST   R2 score: %2.4f, relative error: %2.4f'%(r2, rel))

In [ ]:
#alpha 0.0004776418992889931
#l1 ratio 0.5
#TRAIN  R2 score: 0.4004, relative error: 0.7743
#TEST   R2 score: 0.2506, relative error: 0.8629

In [ ]:
from itertools import combinations
class WHTFeatures:
    def __init__(self, degree):
        self.degree = degree
    def _create_low_degree_support(self, n):
        degree = self.degree
        if degree >= 0:
            support = np.zeros((1, n), dtype=np.int32)
        if degree >= 1:
            support = np.concatenate([support, np.eye(n, dtype=np.int32)])
        if degree >= 2:
            pairs = []
            for i in range(n-1):
                for j in range(i+1, n):
                    pair = np.zeros((1, n), dtype=np.int32)
                    pair[0, i] = 1
                    pair[0, j] = 1
                    pairs += [pair]
            pairs = np.concatenate(pairs, axis=0)
            support = np.concatenate([support, pairs], axis=0)
        if degree >= 3:
            triples = []
            for i, j, k in combinations(np.arange(n), 3):
                triple = np.zeros((1, n), dtype=np.int32)
                triple[0, i] = 1
                triple[0, j] = 1
                triple[0, k] = 1
                triples += [triple]
            triples = np.concatenate(triples, axis=0)
            support = np.concatenate([support, triples], axis=0)
        if degree >= 4:
            quads = []
            for i, j, k, l in combinations(np.arange(n), 4):
                quad = np.zeros((1, n), dtype=np.int32)
                quad[0, i] = 1
                quad[0, j] = 1
                quad[0, k] = 1
                quad[0, l] = 1
                quads += [quad]
            quads = np.concatenate(quads, axis=0)
            support = np.concatenate([support, quads], axis=0)
        if degree > 4:
            raise NotImplementedError("degree higher than 2 is not implemented")
        return support
    
    def fit_transform(self, X):
        n = X.shape[1]
        support = self._create_low_degree_support(n)
        return (-1)**X.dot(support.T)

In [ ]:
feats = WHTFeatures(3)
X_train_poly = feats.fit_transform(X_train_binary)
X_valid_poly = feats.fit_transform(X_valid_binary)
X_test_poly = feats.fit_transform(X_test_binary)
X_train_valid_poly = np.concatenate((X_train_poly, X_valid_poly), axis=0)

est = ElasticNetCV(cv=ps, max_iter=10000)
est.fit(X_train_valid_poly, Y_train_valid)
print('alpha', est.alpha_)
print('l1 ratio', est.l1_ratio_)
Y_pred = est.predict(X_test_poly)
Y_pred2 = est.predict(X_train_poly)
r2_train = r2_score(Y_train, Y_pred2)
rel_train = np.linalg.norm(Y_train - Y_pred2)/np.linalg.norm(Y_train)
r2 = r2_score(Y_test, Y_pred)
rel = np.linalg.norm(Y_test - Y_pred)/np.linalg.norm(Y_test)
print('TRAIN  R2 score: %2.4f, relative error: %2.4f'%(r2_train, rel_train))
print('TEST   R2 score: %2.4f, relative error: %2.4f'%(r2, rel))

In [ ]:
# alpha 0.004687363594863787
# l1 ratio 0.5
# TRAIN  R2 score: 0.4553, relative error: 0.7380
# TEST   R2 score: 0.3383, relative error: 0.8109

In [ ]:
feats = WHTFeatures(2)
X_train_poly = feats.fit_transform(X_train_binary)
X_valid_poly = feats.fit_transform(X_valid_binary)
X_test_poly = feats.fit_transform(X_test_binary)
X_train_valid_poly = np.concatenate((X_train_poly, X_valid_poly), axis=0)

est = ElasticNetCV(cv=ps, max_iter=10000)
est.fit(X_train_valid_poly, Y_train_valid)
print('alpha', est.alpha_)
print('l1 ratio', est.l1_ratio_)
Y_pred = est.predict(X_test_poly)
Y_pred2 = est.predict(X_train_poly)
r2_train = r2_score(Y_train, Y_pred2)
rel_train = np.linalg.norm(Y_train - Y_pred2)/np.linalg.norm(Y_train)
r2 = r2_score(Y_test, Y_pred)
rel = np.linalg.norm(Y_test - Y_pred)/np.linalg.norm(Y_test)
print('TRAIN  R2 score: %2.4f, relative error: %2.4f'%(r2_train, rel_train))
print('TEST   R2 score: %2.4f, relative error: %2.4f'%(r2, rel))

In [ ]:
#alpha 0.001368444642567742
#l1 ratio 0.5
#TRAIN  R2 score: 0.3917, relative error: 0.7799
#TEST   R2 score: 0.2799, relative error: 0.8459

In [ ]:
feats = WHTFeatures(4)
X_train_poly = feats.fit_transform(X_train_binary)
X_valid_poly = feats.fit_transform(X_valid_binary)
X_test_poly = feats.fit_transform(X_test_binary)
X_train_valid_poly = np.concatenate((X_train_poly, X_valid_poly), axis=0)

est = ElasticNetCV(cv=ps, max_iter=10000)
est.fit(X_train_valid_poly, Y_train_valid)
print('alpha', est.alpha_)
print('l1 ratio', est.l1_ratio_)
Y_pred = est.predict(X_test_poly)
Y_pred2 = est.predict(X_train_poly)
r2_train = r2_score(Y_train, Y_pred2)
rel_train = np.linalg.norm(Y_train - Y_pred2)/np.linalg.norm(Y_train)
r2 = r2_score(Y_test, Y_pred)
rel = np.linalg.norm(Y_test - Y_pred)/np.linalg.norm(Y_test)
print('TRAIN  R2 score: %2.4f, relative error: %2.4f'%(r2_train, rel_train))
print('TEST   R2 score: %2.4f, relative error: %2.4f'%(r2, rel))